# LSTM Model

Dies ist das etwas komplexere Model, das wir eigentlich benutzen wollten um die neuen Tweets zu generieren. Es besteht aus einer "Embedding Layer" mit 50 Neuronen, zwei "LSTM Layer" mit je 100 Neuronen und zwei "Fully-Connected Layer", die erste mit 100 Neuronen und die zweite mit so vielen Neuronen wie es Wörter in unserem Vokabular gibt. Bei der Struktur des Models haben wir uns an einem Tutorial orientiert, das ein Word-Level Model in Keras zeigt (https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/). Weitere Erläuterungen und Problembeschreibungen sind als Kommentare im Code eingefügt.

In [2]:
# Importiere die benötigten packages
import numpy as np
import tensorflow as tf
import random
import scipy.io
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import pandas as pd
import re

In [3]:
# Lade die Textdatei ins Notebook. Der Text wird utf-8 encoded.
# Zeichen, die von dieser Codierung nicht verstanden werden, werden erst einmal ignoriert.
with open("tweets/LINKE.txt", encoding="utf8", errors='ignore') as f:
    textLINKE = f.read()   

In [4]:
''' Diese Funktion entfernt alle Zeichen, die wir nicht haben wollen aus der Textdatei. Da die Tweets zunächst als Listenelemente
    aneinander gereit wurden (siehe Preprocessing), sind in der Textdatei noch sehr viele eckige Klammern. Außerdem wurden 
    Zeilenumbrüche als '/n' codiert und Tweets, die die Standardlänge von 140 Zeichen überschritten haben, wurden abgeschnitten,
    sodass teilweise unvollständige Worte vorkommen. Zudem gibt es noch einige weitere ungewünschte Zeichen, wie URL-links etc.
    Input:  Eine String Datei
    Output: Die Datei ohne die unerwünschten Zeichen
'''
def clean_text(text):
    
    # entfernt URL Links
    text = re.sub(r"http\S+", "", text)
    # entfernt @screen_name
    text = re.sub(r"@\S+", "", text)
    # entfernt "RT" für Re-Tweet
    text = re.sub(r"RT", "", text)
    # entfernt Sondersymbole wie Emojis
    text = re.sub(r"🇩🇪", "", text)
    #text = re.sub(r"😁", "", text)
    #text = re.sub(r"😀", "", text)
    #text = re.sub(r"'❤'", "", text)
    #text = re.sub(r"❤", "", text)
    #text = re.sub(r"🎉", "", text)
    # entfernt Sonderzeichen
    text = re.sub(r"'️", "", text)
    text = re.sub(r"…", "", text)
    text = re.sub(r"\\n", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"]\S+", "", text)
    # entfernt einzelne Symbole 
    text = ''.join( c for c in text if  c not in '[,],/,:,&,_,1,2,3,4,5,6,7,8,9,0,,' )

    # ersetzt Umlaute und ß
    text = re.sub(r"ü", "ue", text)
    text = re.sub(r"ö", "oe", text)
    text = re.sub(r"ä", "ae", text)
    text = re.sub(r"ß", "ss", text)
    
    # setzt ein Leerzeichen vor jeden Punkt, sodass er als einzelnes Symbol gesehen wird und Wörter mit Punkt
    # nicht als eigenständige Wörter ins Vokabular eingehen
    text = re.sub(r"\.", " .", text)
    
    return text

In [7]:
# Benutze die clean_text Funktion um den Text von ungewollten Symbolen zu befreien
text = clean_text(textLINKE)

# Wir haben verschiede Versionen ausprobiert um den Text zu tokenisieren
# Sie haben alle ihre Vor- und Nachteile, manche beschleunigen das Training, aber dafür streichen sie alle Sonderzeichen,
# andere wiederum sind langsamer, weil alle Sonderzeichen als einzelne "Wörter" interpretiert werden etc

# 1. Möglichkeit
tokenized_text = list(text.split(" "))
vocab = set(tokenized_text)

# 2. Möglichkeit - tokenisiert den Text, aber löscht einzelne Buchstaben und Satzzeichen
#tokenizer = RegexpTokenizer(r'\w+')
#tokenized_text = list(tokenizer.tokenize(text_char))
#vocab = set(tokenized_text)

# Speicher die Länge des Vokabulars und des Textes als einzelne Variablen ab um später darauf zugreifen zu können
vocab_size = len(vocab)
print("vocab size: {}".format(vocab_size))

text_len = len(tokenized_text)
print("text lenght: {}".format(text_len))

vocab size: 2596
text lenght: 7379


In [51]:
# Kreiere Dictionaries um den Text in Zahlen umwandeln zu können 
# und später die Zahlen wieder in Text
# Gehe einmal durch ganze Vokabular und "zähle mit", sodass jedem Wort eine Zahl zugeordnet wird
word_to_id = {word:i for i, word in enumerate(vocab)}
id_to_word = {i:word for i, word in enumerate(vocab)}

# Übersetze den Text zu den zugehörigen IDs
text_idx = [word_to_id[word] for word in tokenized_text]


## Generieren des Datensets

In [56]:
# Bestimme die Länge der einzelnen Sequenzen.
# Wir haben uns erstmal für eine Länge von 20 Wörtern entschieden, da eine längere Abhängigkeit von Wörtern in dem Kontext von
# Tweets unserer Meinung nach wenig Sinn macht
# Zu jeder Sequenz gehört ein Targetwort, welches das nächste Wort in der Reihe ist. Daher addiert man zu der Länge noch 1
seq_len = 20+1
# Erstelle eine leere Liste zum abspeichern der Sequenzen
sequences = []
# Starte vorne in der Liste und speicher die ersten 21 IDs als erste Sequenz, gehe zur zweiten ID usw.
# Die letzte Sequenz endet mit dem letzten Wort als Target, dh das Wort an Stelle text_len - seq_len ist der Beginn der letzten Sequenz
for i in range(seq_len, len(text_idx)):
    # wähle die Sequenz von IDs aus
    seq = text_idx[i-seq_len:i]
    # und speicher sie in der Liste ab
    sequences.append(seq)
print('Total Sequences: {}'.format(len(sequences)))

Total Sequences: 7358


In [57]:
# Zerteile die Sequenzen in Inputs der Länge seq_len und Target der Länge 1
# Dazu mache die Liste erstmal zu einem Array
sequences = np.array(sequences)
# Nun zerteile die einzelnen Sequenzen
input_data, target = sequences[:,:-1], sequences[:,-1]

# Speicher die seq_length nocheinal dynamisch als die Länge der einzelnen Input Sequenzen ab, damit sie sich automatisch ändert,
# wenn der Input sich ändert
seq_length = input_data.shape[1]

In [58]:
# Jetzt fangen wir an das Model zu bauen
# Dafür brauchen wir ersteinmal ein Tensor Datenset

# Wie immer resetten wir den default graph
tf.reset_default_graph()

# Der Targetvektor wird one-hot encoded, damit er später mit dem Output der Softmax Funktion verglichen werden kann
# Dafür muss er zunächst zu dem richtigen Datentyp gecastet werden. Das ist hier kein Problem, weil die IDs alle ganze Zahlen sind
# und damit leicht als Integer statt als Float gespeichert werden können
target = tf.cast(target, dtype = tf.int32)
target = tf.one_hot(target, depth=vocab_size)
# Der Input wird ebenfalls als Integer gespeichert, aber nicht als one-hot
input_data = tf.cast(input_data, dtype = tf.int32)

# Kreiere das Tensorflow Datenset mit den Input Sequenzen und dem one-hot Target
dataset = tf.data.Dataset.from_tensor_slices((input_data,target))

# Wähle ein Batchsize und zerteile das Datenset in Batches
batchsize = 128
dataset = dataset.batch(batchsize)

# Initialisiere den Iterator wie gehabt
iterator = tf.data.Iterator.from_structure(dataset.output_types,dataset.output_shapes)
iterator_init_op = iterator.make_initializer(dataset)

# Generiere den Input Batch und den zugehörigen Target Batch mit Hilfe von .get_next()
next_batch = iterator.get_next()
input_data = next_batch[0]
target_data = next_batch[1]

# Initialisiere die Placeholder für den state der zwei LSTM layer
# Der initial_state soll zunächst aus Nullen bestehen und dann immer wieder an das Model zurück gegeben werden
# Dies passiert später im sess.run() mit der Hilfe von feed.dict{}
# Wähle die Größe der LSTM layer
lstm_size = 100
lstm_state1 = tf.placeholder(shape=[seq_length, lstm_size], dtype=np.float32)
lstm_state2 = tf.placeholder(shape=[seq_length, lstm_size], dtype=np.float32)


## Das Model

In [59]:
# Definiere das Model
# Zunächst wollen wir eine Embedding Layer, die die Embedding Vektoren der Input Wörter zurück gibt und die EMbedding Matrix trainiert
with tf.variable_scope("embedding", reuse=tf.AUTO_REUSE):
    embedding = tf.contrib.layers.embed_sequence(
        ids = input_data,
        vocab_size = vocab_size,
        embed_dim = 50,
        unique = False,
        trainable = True)
    print(embedding)

# Danach folgen zwei LSTM Layer mit jeweils 100 Neuronen
with tf.variable_scope("LSTM1", reuse=tf.AUTO_REUSE):
    cell = tf.nn.rnn_cell.LSTMCell(
        num_units = lstm_size,
        forget_bias=1.0,
        state_is_tuple=True,
        activation='tanh')
    # Der Initial_state wird zunächst auf Null gesetzt
    # Hier soll eigentlich der Placeholder reingegeben werden, aber der einfachheitshalber haben wir es ersteinmal ohne probiert
    initial_state = cell.zero_state(seq_length, dtype=tf.float32)
    #initial_state = lstm_state1
    
    # Die Sequenzen im Input Batch werden einzelnen in die LSTM Zelle eingespeist und verarbeitet 
    lstm1 = []
    for batch in range(batchsize):
        lstm_outputs, final_state = cell(embedding[batch], initial_state)
        lstm1.append(lstm_outputs)
    # Der letzte final_state, in dem sich die Zelle befindet, wird abgespeichert und im nächsten Batch eingelesen
    remember1 = final_state
    print(len(lstm1))
    print(lstm1[0])
    print(remember1)
    
# Die zweite LSTM Layer ist aufegabut wie die erste
with tf.variable_scope("LSTM2", reuse=tf.AUTO_REUSE):
    cell = tf.nn.rnn_cell.LSTMCell(
        num_units = lstm_size,
        forget_bias=1.0,
        state_is_tuple=True,
        activation='tanh')
    initial_state = cell.zero_state(seq_length, tf.float32)
    #initial_state = lstm_state2
    lstm2 = []
    for batch in lstm1:
        lstm_outputs, final_state = cell(batch, initial_state)
        lstm2.append(lstm_outputs)
    remember2 = final_state
    print(len(lstm2))
    print(lstm2[0])
    print(remember2)
    
# Die erste fully-connected layer besteht aus 100 Neuronen und benutzt die ReLU als activation function    
with tf.variable_scope("fully_connected1", reuse=tf.AUTO_REUSE):
    full = tf.contrib.layers.fully_connected(
        lstm2,
        100,
        activation_fn = tf.nn.relu)
    print(full)

# Die zweite fully-connected layer hat so viele Neuronen, wie es Wörter in dem aktuellen Vokabular gibt.
# Die activation function ist nun die Softmax, sodass wir den Output des Models mit dem Target vergleichen können
with tf.variable_scope("fully_connected2", reuse=tf.AUTO_REUSE):
    logits = tf.contrib.layers.fully_connected(
        full,
        vocab_size,
        activation_fn = tf.nn.softmax)
    print(logits)

Tensor("embedding/EmbedSequence/embedding_lookup/Identity:0", shape=(?, 20, 50), dtype=float32)
128
Tensor("LSTM1/lstm_cell/mul_2:0", shape=(20, 100), dtype=float32)
LSTMStateTuple(c=<tf.Tensor 'LSTM1/lstm_cell/add_255:0' shape=(20, 100) dtype=float32>, h=<tf.Tensor 'LSTM1/lstm_cell/mul_383:0' shape=(20, 100) dtype=float32>)
128
Tensor("LSTM2/lstm_cell/mul_2:0", shape=(20, 100), dtype=float32)
LSTMStateTuple(c=<tf.Tensor 'LSTM2/lstm_cell/add_255:0' shape=(20, 100) dtype=float32>, h=<tf.Tensor 'LSTM2/lstm_cell/mul_383:0' shape=(20, 100) dtype=float32>)
Tensor("fully_connected1/fully_connected/Relu:0", shape=(128, 20, 100), dtype=float32)
Tensor("fully_connected2/fully_connected/Softmax:0", shape=(128, 20, 2596), dtype=float32)


### Problembeschreibung

Das Model läuft zwar bis zu diesem Zeitpunkt ohne Fehlermeldung, das Problem ist aber, dass die Output Dimensionen nicht stimmen. Bis zur zweiten LSTM Layer passt noch alles - die Embedding Layer gibt einen Ouput mit Dimensionen (batch_size, seq_length, embed_size) weiter und die erste LSTM Layer produziert eine Liste mit der Länge batch_size, in der jeder Eintrag die Dimensionen (seq_length, lstm_size) hat. Die zweite LSTM Layer prdoziert einen Output mit den gleichen Dimensionen, sollte aber eigentlich nur die Dimensionen (batch_size, lstm_size) haben. Diese falsche "Extra-Dimension" zieht sich dann natürlich auch durch die beiden fully-connectes Layer, sodass die Logits am Ende eine Größe von (seq_length * batch_size, vocab_size) statt nur (batch_size, vocab_size) haben. Vermutlich kommt der Fehler daher, dass die LSTM Layer eine Liste produziert statt eines Tensors. Aber auch nach viel rumexperimenieren und nachlesen konnten wir noch keine Lösung für das Problem finden.

### Definition von Training Loss und der Optimierungsstrategie

In [60]:
# Der loss wird durch die cross entropy bestimmt, in der der Output des Models mit dem tatsächlichen Target verglichen wird
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=target_data, logits=logits))

In [61]:
# Zum Optimieren benutzen wir den Adams Optimizer und eine Lernrate von 0.001
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-3)
training_step = optimizer.minimize(loss)

### Problembeschreibung

Hier kommt unser zweites Problem auf. Wir schaffen es immer noch nicht die Placeholder korrekt einzulesen.
Deshalb haben wir zwei Sessions geschrieben. Diese ist so wie wie wir uns das im Idealfall vorgestellt haben. Der final_state in jeder LSTM Layer wird in der Variabel "remember1" bzw "remember2" gespeichert und ausgelesen und anschließend wieder in den Placeholder eingespeist, sodass der nächste Batch den final_state des vorherigen Batches als initial_state hat.

Die zweite eine Zelle weiter unten ist die, mit der wir zunächste gearbeitet haben um das Model zu bauen. Sie hat keine Placeholder ist aber ansonsten gleich. In der unteren kann man den Fehler sehen, den wir oben beschrieben haben. Nämlich das die Logits die falsche Shape haben.


## Session Nr.1

In [49]:
with tf.Session() as sess:
    # wir starten damit die Variablen zu initialisieren
    sess.run(tf.global_variables_initializer())
    
    # bestimme die Anzahl der Epochen
    for epoch in range(100):
    
        # initialisiere einen Step counter um an den richtigen Stellen die Placeholder einzuspeisen
        global_step = 0
        # lade das Datenset in den Iterator
        sess.run(iterator_init_op)

        # gehe durch das Datenset bis es leer ist
        while True:
            try:
                # Falls dies der erste Step ist, initialisiere die initial_states der beiden LSTM Layer mit Nullen 
                if global_step == 0:
                    state1 = np.zeros([seq_length,lstm_size], dtype = np.float32)
                    state2 = np.zeros([seq_length,lstm_size], dtype = np.float32)

                # Starte das Training und gebe den loss aus
                # außerdem gebe die states der beiden LSTM Layer aus um sie für den nächsten Batch zu benutzen
                loss_val, state1, state2, _ = sess.run([loss, remember1, remember2, training_step],
                                                      feed_dict = {lstm_state1:state1, lstm_state2:state2})
                
                # Erhöhe den Global Step Count um 1
                global_step += 1

            # Stoppt sobald der Iterator leer ist
            except tf.errors.OutOfRangeError:
                break
                
       # Nach jeder Epoche soll der loss ausgegeben werden um den Trainingsprozess zu überwachen
        print("Epoch: {}, Loss: {:f}".format(epoch, loss_val))
        
        # Wir wollen schon während des Trainings Tweets generieren um zu schauen, wie sich die Qualität verändert
        
        # Bestimme die Länge der generierten Tweets
        tweet_length = 20
        
        # Wähle zufällig eine Startsequenz aus
        start_idx = random.randint(0, len(text_idx) - seq_len)
        seq_idx = text_idx[start_idx:start_idx + seq_len]      
      
        # Erstelle eine leere Liste um die generierten Wörter abzuspeichern
        sample_seq_idx = []
        
        # Generiere Wort für Wort einen euen Tweet
        for n in range(tweet_length):
            
            # Da wir die generierten Tweets nur anschauen wollen und nicht mit einem Target vergleichen, brauchen wir Faketargets
            fake_target = np.zeros([1,tweet_length], dtype=np.int32)
            sample_dataset = tf.data.Dataset.from_tensor_slices(([seq_idx], fake_target))
            # Lade das Sample Datenset in den Iterator
            sess.run(iterator.make_initializer(sample_dataset))
    
            # Lese den Softmax Output der letzten Layer aus (dieser liefert die Wahrscheinlichkeiten für das nächste generierte Wort)
            # und gebe den state der LSTM Layer immer wieder neu rein
            sample_output, state1, state2 = sess.run([logits, remember1, remember2],
                                                     feed_dict={lstm_state1: state1, lstm_state2:state2})

            
            # Wähle zufällig ein Wort aus der Softmax Distribution und hänge es an die Liste an
            sample = np.random.choice(range(vocab_size), p=sample_output.ravel())
            sample_seq_idx.append(sample)
            # Update die Startsequenz mit dem neuen Wort um das nächste Wort zu generieren
            seq_idx = seq_idx[1:] + [sample]
        
      
        # Zeige den generierten Tweets an
        # da wir in dieser Tokenisierung keine Leerzeichen haben, muss zwischen jedes Wort eins gesetzt werden
        sample_txt = ' '.join(idx_to_char[idx] for idx in sample_seq_idx)
        print('----\n %s \n----\n' % (sample_txt,))            

TypeError: unhashable type: 'numpy.ndarray'

## Session Nr.2 

In [62]:
with tf.Session() as sess:
    # wir starten damit die Variablen zu initialisieren
    sess.run(tf.global_variables_initializer())
    
    # bestimme die Anzahl der Epochen
    for epoch in range(100):
    
        # initialisiere einen Step counter um an den richtigen Stellen die Placeholder einzuspeisen
        global_step = 0
        # lade das Datenset in den Iterator
        sess.run(iterator_init_op)

        # gehe durch das Datenset bis es leer ist
        while True:
            try:

                # Starte das Training und gebe den loss aus
                # außerdem gebe die states der beiden LSTM Layer aus um sie für den nächsten Batch zu benutzen
                loss_val, _ = sess.run((loss, training_step))
                
                # Erhöhe den Global Step Count um 1
                global_step += 1

            # Stoppt sobald der Iterator leer ist
            except tf.errors.OutOfRangeError:
                break
                
       # Nach jeder Epoche soll der loss ausgegeben werden um den Trainingsprozess zu überwachen
        print("Epoch: {}, Loss: {:f}".format(epoch, loss_val))
        
        # Wir wollen schon während des Trainings Tweets generieren um zu schauen, wie sich die Qualität verändert
        
        # Bestimme die Länge der generierten Tweets
        tweet_length = 20
        
        # Wähle zufällig eine Startsequenz aus
        start_idx = random.randint(0, len(text_idx) - seq_len)
        seq_idx = text_idx[start_idx:start_idx + seq_len]      
      
        # Erstelle eine leere Liste um die generierten Wörter abzuspeichern
        sample_seq_idx = []
        
        # Generiere Wort für Wort einen euen Tweet
        for n in range(tweet_length):
            
            # Da wir die generierten Tweets nur anschauen wollen und nicht mit einem Target vergleichen, brauchen wir Faketargets
            fake_target = np.zeros([1, vocab_size], dtype=np.int32)
            sample_dataset = tf.data.Dataset.from_tensor_slices(([seq_idx], fake_target))
            # Lade das Sample Datenset in den Iterator
            sess.run(iterator.make_initializer(sample_dataset))
    
            # Lese den Softmax Output der letzten Layer aus (dieser liefert die Wahrscheinlichkeiten für das nächste generierte Wort)
            # und gebe den state der LSTM Layer immer wieder neu rein
            sample_output = sess.run((logits))

            # Wähle zufällig ein Wort aus der Softmax Distribution und hänge es an die Liste an
            sample = np.random.choice(range(vocab_size), p=sample_output.ravel())
            sample_seq_idx.append(sample)
            # Update die Startsequenz mit dem neuen Wort um das nächste Wort zu generieren
            seq_idx = seq_idx[1:] + [sample]
        
      
        # Zeige den generierten Tweets an
        # da wir in dieser Tokenisierung keine Leerzeichen haben, muss zwischen jedes Wort eins gesetzt werden
        sample_txt = ' '.join(idx_to_char[idx] for idx in sample_seq_idx)
        print('----\n %s \n----\n' % (sample_txt,))            

InvalidArgumentError: logits and labels must be broadcastable: logits_size=[2560,2596] labels_size=[128,2596]
	 [[{{node softmax_cross_entropy_with_logits}} = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](softmax_cross_entropy_with_logits/Reshape, IteratorGetNext:1)]]

Caused by op 'softmax_cross_entropy_with_logits', defined at:
  File "C:\Users\Sophia\Anaconda3\envs\tfcourse\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Sophia\Anaconda3\envs\tfcourse\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Sophia\Anaconda3\envs\tfcourse\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Sophia\Anaconda3\envs\tfcourse\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Sophia\Anaconda3\envs\tfcourse\lib\site-packages\ipykernel\kernelapp.py", line